<a href="https://colab.research.google.com/github/anskong/ai_ref_from_lessons/blob/main/LGCNS_LangChain_010.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# LCEL 번역 체인 예제

In [ ]:
# 1) 필수 패키지 설치
!pip install --quiet --upgrade "langchain-core>=0.3.0" "langchain-openai>=0.3.0" openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 438.3/438.3 kB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.4/63.4 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 720.4/720.4 kB 17.7 MB/s eta 0:00:00


In [ ]:
# 2) OPENAI API 키 로드
import os
from google.colab import userdata

# API 키 등록
os.environ["OPENAI_API_KEY"] = userdata.get("OPENAI_API_KEY")
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

# 3) LangChain 컴포넌트 임포트
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser

# 4) 프롬프트 템플릿 정의
translation_prompt = PromptTemplate.from_template(
    "Translate this to {language}:\n\n{input}"
)

# 5) 모델 및 출력 파서 정의
llm = ChatOpenAI(
    model="gpt-4.1-nano",  # 또는 gpt-4o-mini
    openai_api_key=OPENAI_API_KEY
)
parser = StrOutputParser()

# 6) LCEL 체인 구성
translation_chain = translation_prompt | llm | parser

# 7) 체인 실행
result = translation_chain.invoke({
    "input": "안녕하세요",
    "language": "English"
})

# 8) 출력
print("번역 결과:", result)

번역 결과: Hello


# 시퀀스 체인 (Sequence Chain) 예
- 실행흐름: PromptTemplate → LLM → Parser
- 각 구성 요소는 Runnable 인터페이스를 만족하고, | 연산자를 통해 연결됨

In [ ]:
# 1) 필수 패키지 설치
!pip install --quiet --upgrade "langchain-core>=0.3.0" "langchain-openai>=0.3.0" openai

In [ ]:
# 2) OPENAI API 키 로드 (Colab의 Tools → Secrets에 등록되어 있어야 함)
import os
from google.colab import userdata

# API 키 등록
os.environ["OPENAI_API_KEY"] = userdata.get("OPENAI_API_KEY")
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

# 3) 필요한 모듈 임포트
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser

# 4) 체인 구성 요소 정의
prompt = PromptTemplate.from_template("Translate to {language}:\n{input}")
llm = ChatOpenAI(model="gpt-3.5-turbo", openai_api_key=OPENAI_API_KEY)
parser = StrOutputParser()

# 5) 시퀀스 체인 구성 (LCEL)
sequence_chain = prompt | llm | parser

# 6) 실행 예시
result = sequence_chain.invoke({"input": "안녕하세요", "language": "English"})

# 7) 출력 결과
print("번역 결과:", result)

번역 결과: Hello


# RunnableParallel을 이용한 사전형 출력 생성


In [ ]:
# 1) 필수 패키지 설치
!pip install --quiet --upgrade "langchain-core>=0.3.0" "langchain-openai>=0.3.0" openai

In [ ]:
# 2) Colab에서 API 키 불러오기 (Tools → Secrets에 등록 필요)
import os
from google.colab import userdata

# API 키 등록
os.environ["OPENAI_API_KEY"] = userdata.get("OPENAI_API_KEY")
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

# 3) 모듈 임포트
from langchain_openai import ChatOpenAI
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

# 4) 프롬프트 정의
translation_prompt = PromptTemplate.from_template(
    "다음 문장을 {language}로 번역하고, 번역된 문장만 대답하세요.\n"
    "```\n{input}\n```"
)

# 5) 모델 및 파서 정의
llm = ChatOpenAI(model="gpt-4.1-nano", openai_api_key=OPENAI_API_KEY)
parser = StrOutputParser()

# 6) 번역 체인 정의
translation_chain = translation_prompt | llm | parser

to_english = {
    "input": RunnablePassthrough(),
    "language": lambda _: "English",
} | translation_chain

to_korean = {
    "input": RunnablePassthrough(),
    "language": lambda _: "Korean",
} | translation_chain

# 7) 전체 체인 구성: 입력 → 영어 번역 → 다시 한국어 번역
full_chain = to_english | llm | parser | to_korean

# 8) 테스트 입력 및 실행
text = "LangChain은 무엇인가요?"
result = full_chain.invoke(text)

# 9) 결과 출력
print("최종 응답:", result)

최종 응답: LangChain은 대형 언어 모델(LLM)을 활용하는 애플리케이션 개발을 용이하게 하는 오픈소스 프레임워크입니다. 이 프레임워크는 LLM과 기타 데이터 소스, API, 워크플로우를 통합하는 것을 간소화하는 도구와 추상화를 제공합니다. LangChain의 주요 목표는 개발자가 챗봇, 질문 답변 시스템, 가상 비서와 같은 복잡하고 맥락을 인식하며 다단계인 AI 애플리케이션을 더 효율적으로 구축할 수 있도록 돕는 것입니다.

LangChain의 핵심 기능은 다음과 같습니다:

- **프롬프트 관리:** 효과적인 프롬프트 설계, 관리, 실행 도구.
- **체인:** 여러 작업 또는 모델을 연결하여 복잡한 작업을 수행하는 모듈형 파이프라인.
- **메모리:** 상호작용 간 대화 맥락 또는 상태를 유지하는 기능.
- **에이전트:** 사용자 입력에 따라 어떤 조치 또는 도구를 호출할지 동적으로 결정하는 구성 요소.
- **통합:** 다양한 데이터 소스, API, 외부 도구와의 연결을 지원하여 AI 능력을 강화.

전반적으로, LangChain은 개발자가 대형 언어 모델의 힘을 활용하여 확장 가능하고 유지보수하기 쉬우며 지능적인 애플리케이션을 구축할 수 있도록 지원하는 것을 목표로 합니다.


# RunnableParallel을 이용한 체인병렬 실행 (입력 언어 판별 → 영어 번역 → 원래 언어로 재번역)

In [ ]:
# 1) 필수 패키지 설치
!pip install --quiet --upgrade "langchain-core>=0.3.0" "langchain-openai>=0.3.0" openai pydantic

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.2/67.2 kB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 444.2/444.2 kB 9.9 MB/s eta 0:00:00


In [ ]:
# 2) OPENAI_API_KEY 로드
import os
from google.colab import userdata

# API 키 등록
os.environ["OPENAI_API_KEY"] = userdata.get("OPENAI_API_KEY")
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

# 3) 모듈 임포트
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import RunnablePassthrough
from pydantic import BaseModel, Field

# 4) LLM 정의
llm = ChatOpenAI(model="gpt-4o-mini", openai_api_key=OPENAI_API_KEY)

# 5) 언어 판별용 구조화 출력 모델 정의
class Language(BaseModel):
    language_name: str = Field(description="언어명 (예: 'Korean')")

llm_with_language_output = llm.with_structured_output(Language)

# 6) 언어 판별 프롬프트 및 체인
ask_language_prompt = PromptTemplate.from_template(
    "다음 문장이 작성된 언어명은 무엇입니까?\n```\n{input}\n```"
)
get_language_chain = ask_language_prompt | llm_with_language_output

# 7) 번역용 프롬프트 및 체인
translation_prompt = PromptTemplate.from_template(
    "다음 문장을 {language}으로 번역하고, 번역된 문장만 대답하십시오.\n```\n{input}\n```"
)
translation_chain = translation_prompt | llm | StrOutputParser()

# 8) 중간 번역: 입력 → 영어 번역
to_english = {
    "input": RunnablePassthrough(),
    "language": lambda _: "English",
} | translation_chain

# 9) 전체 체인 구성: 원문 → 영어 번역 → 원래 언어 판별 → 다시 번역
full_chain = {
    "input": to_english | llm | StrOutputParser(),
    "language": get_language_chain | (lambda x: x.language_name),
} | translation_chain

# 10) 예제 실행
text = "LangChain은 무엇인가요?"
result = full_chain.invoke(text)

# 11) 결과 출력
print("최종 응답:", result)

최종 응답: LangChain은 대규모 언어 모델(LLM)을 활용한 애플리케이션 개발을 위해 설계된 오픈 소스 프레임워크입니다. LLM을 이용한 자연어 이해, 텍스트 생성, 대화형 에이전트와 같은 작업을 위한 복잡한 애플리케이션을 만들기 위한 도구와 구성 요소를 개발자에게 제공합니다.

LangChain의 주요 기능은 다음과 같습니다:

1. **모듈형 구성 요소**: 텍스트 처리, 메모리 관리 및 체인 작업과 같은 다양한 기능을 쉽게 통합할 수 있는 모듈형 구성 요소 세트를 제공합니다.

2. **사고의 연쇄**: LangChain은 한 모델의 출력을 다른 모델에 공급할 수 있는 "체인" 만들기를 지원하여 애플리케이션 내에서 더 복잡한 워크플로우와 논리를 가능하게 합니다.

3. **외부 도구와의 통합**: 이 프레임워크는 다양한 API, 데이터베이스 및 기타 서비스와의 통합을 허용하여 LLM으로 구축된 애플리케이션의 기능을 향상시킵니다.

4. **메모리 지원**: LangChain은 대화나 다단계 작업에 대한 맥락을 유지할 수 있어 상호작용적이고 반응적인 애플리케이션을 만드는 데 필수적입니다.

5. **프롬프트 관리**: 언어 모델에 전송되는 프롬프트를 관리하고 최적화하기 위한 도구를 제공하여 성능과 결과를 크게 향상시킬 수 있습니다.

LangChain은 고급 언어 처리 기능이 필요한 챗봇, 콘텐츠 생성 도구 및 다양한 AI 기반 애플리케이션을 구축하려는 개발자에게 특히 유용합니다. 그 유연성과 사용 편의성 덕분에 AI 및 자연어 처리를 다루는 연구자와 산업 전문가 모두에게 매력적인 선택이 됩니다.
